In [1]:
import pandas as pd

subset_method = "union"
correlation_type = "pearson"
data_category = pd.read_feather("../../all_data/xwas/univariate_correlations/correlations/categories/correlations_UrineBiochemistry.feather").to_dict()

In [2]:
correlations_raw = pd.DataFrame(data_category).set_index(["dimension_1", "subdimension_1", "r2_1", "r2_std_1", "dimension_2", "subdimension_2", "r2_2", "r2_std_2"])
correlations_raw.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations_raw.columns.tolist())), names=["subset_method", "correlation_type"]
)
correlations = correlations_raw[[(subset_method, correlation_type)]]
correlations.columns = ["correlation"]
correlations.reset_index(inplace=True)

In [7]:
correlations["dimension_1"].drop_dupicates()

AttributeError: 'Series' object has no attribute 'drop_dupicates'

In [4]:
correlations.pivot(
    index=["dimension_1", "subdimension_1"],
    columns=["dimension_2", "subdimension_2"],
    values="correlation",
)

dimension_2                          Abdomen                      Arterial  \
subdimension_2                             *     Liver  Pancreas         *   
dimension_1      subdimension_1                                              
NaN              NaN                0.714784  0.815369  0.618678 -0.999968   
*                *                  0.993278  0.993778  0.970452 -0.739614   
*instances01     *                  0.999768  0.984097  0.994203  1.000000   
*instances1.5x   *                       NaN       NaN  1.000000 -1.000000   
*instances23     *                       NaN       NaN  1.000000       NaN   
Abdomen          *                       NaN       NaN  1.000000 -1.000000   
                 Liver                   NaN       NaN  1.000000 -1.000000   
                 Pancreas           1.000000  1.000000  1.000000 -1.000000   
Arterial         *                 -1.000000 -1.000000 -1.000000       NaN   
                 Carotids                NaN       NaN -1.000000       NaN   
                 PulseWaveAnalysis  0.663708  0.595427  0.749696  0.096964   
Biochemistry     *                  0.780192  0.786578  0.712636 -0.863915   
                 Blood             -0.711016 -0.635425 -0.703366  0.461412   
                 Urine              0.773851  0.769666  0.711907 -0.825580   
Brain            *                  0.861445  0.808252  0.917053 -0.200515   
                 Cognitive               NaN       NaN -1.000000       NaN   
                 MRI                0.920119  0.897805  0.952052 -0.378459   
Eyes             *                  0.893760  0.858904  0.936991 -0.295088   
                 All                0.515277  0.470247  0.621050  0.148969   
                 Fundus             0.930642  0.928202  0.947819 -0.473519   
                 OCT                0.121254  0.227223 -0.006280 -0.823997   
Hearing          *                  0.599434  0.557220  0.697244  0.048044   
Heart            *                       NaN       NaN  1.000000 -1.000000   
                 ECG                     NaN       NaN  1.000000 -1.000000   
                 MRI                     NaN       NaN  1.000000 -1.000000   
Lungs            *                  0.282904  0.209783  0.395320  0.481928   
Musculoskeletal  *                  1.000000  1.000000  0.995652 -0.536023   
                 FullBody           1.000000  1.000000  0.984814 -0.465769   
                 Hips               1.000000  1.000000  1.000000 -1.000000   
                 Knees              0.954706  0.953176  0.916648 -0.284358   
                 Scalars           -1.000000 -1.000000 -1.000000  1.000000   
                 Spine             -1.000000 -1.000000 -1.000000  1.000000   
PhysicalActivity *                       NaN       NaN  1.000000 -1.000000   

dimension_2                                                    Biochemistry  \
subdimension_2                      Carotids PulseWaveAnalysis            *   
dimension_1      subdimension_1                                               
NaN              NaN               -0.902571          0.418310     0.449112   
*                *                 -0.962527          0.574074     0.835322   
*instances01     *                  1.000000          0.466502     0.919198   
*instances1.5x   *                       NaN          0.454833     0.834465   
*instances23     *                       NaN          0.843615     0.607427   
Abdomen          *                       NaN          0.663708     0.780192   
                 Liver                   NaN          0.595427     0.786578   
                 Pancreas          -1.000000          0.749696     0.712636   
Arterial         *                       NaN          0.096964    -0.863915   
                 Carotids                NaN         -0.531373    -0.743398   
                 PulseWaveAnalysis -0.531373          1.000000     0.087630   
Biochemistry     *                 -0.743398          0.087630     1.000000   
      

In [6]:
table_correlations = correlations.pivot(
    index=["dimension_1", "subdimension_1"],
    columns=["dimension_2", "subdimension_2"],
    values="correlation",
)

customdata_list = []
for customdata_item in ["r2_1", "r2_std_1", "r2_2", "r2_std_2"]:
    customdata_list.append(
        correlations.pivot(
            index=["dimension_1", "subdimension_1"],
            columns=["dimension_2", "subdimension_2"],
            values=customdata_item,
        ).values
    )
stacked_customdata = list(map(list, np.dstack(customdata_list)))


ValueError: Index contains duplicate entries, cannot reshape